In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("Data/train.csv", index_col="PassengerId")
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Exploratory Data Analysis (EDA)

In [14]:
# split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Survived'])
valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['Survived'])

In [18]:
print('Train data traget distribution:', train_df.Survived.value_counts(), end='\n\n')
print('Validation data traget distribution:', valid_df.Survived.value_counts(), end='\n\n')
print('Test data traget distribution:', test_df.Survived.value_counts())


Train data traget distribution: Survived
0    439
1    273
Name: count, dtype: int64

Validation data traget distribution: Survived
0    55
1    34
Name: count, dtype: int64

Test data traget distribution: Survived
0    55
1    35
Name: count, dtype: int64


In [19]:
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0000,NaN,S
528,0,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S
856,1,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.3500,NaN,S
802,1,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S
